In [11]:
# %pip install pandas
# %pip install matplotlib

In [12]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import os

In [13]:
LOG_FILE = "a2c_logs_20251011_173431.txt"

In [14]:
def parse_logs(log_file_path):
    """
    Parses the detailed training log file to extract key metrics for each episode.

    Args:
        log_file_path (str): The path to the logs.txt file.

    Returns:
        pandas.DataFrame: A DataFrame containing the metrics for each episode.
    """
    
    episode_data = []
    
    metrics_pattern = re.compile(
        r"Episodes: (\d+) \| Win Rate: ([\d.]+)% \| .*? Left Click Avg: ([\d.]+) c/g .*? Right Click Avg: ([\d.]+) c/g .*? Total Click Avg: ([\d.]+) c/g .*? Game Conclusion History: (\d+) Wins, (\d+) Losses"
    )

    with open(log_file_path, 'r') as f:
        content = f.read()

    # Find all summary blocks
    summaries = content.split('============================================================')
    
    for summary in summaries:
        if "Episode" not in summary or "finished" not in summary:
            continue
            
        metrics_match = metrics_pattern.search(summary)
        if metrics_match:
            episode = int(metrics_match.group(1))
            win_rate = float(metrics_match.group(2))
            left_click_avg = float(metrics_match.group(3))
            right_click_avg = float(metrics_match.group(4))
            total_click_avg = float(metrics_match.group(5))
            wins = int(metrics_match.group(6))
            losses = int(metrics_match.group(7))
            
            episode_data.append({
                "Episode": episode,
                "Win Rate (%)": win_rate,
                "Wins": wins,
                "Losses": losses,
                "Left Click Avg (c/g)": left_click_avg,
                "Right Click Avg (c/g)": right_click_avg,
                "Total Click Avg (c/g)": total_click_avg
            })

    return pd.DataFrame(episode_data)

def generate_graphs(df, save_dir="./graphs"):
    """
    Generates and saves plots for the training metrics.
    """
    if df.empty:
        print("DataFrame is empty, cannot generate graphs.")
        return

    if LOG_FILE.startswith("a2c"):
        save_dir = "./graphs/a2c"
    elif LOG_FILE.startswith("dqn"):
        save_dir = "./graphs/dqn"
    elif LOG_FILE.startswith("mc"):
        save_dir = "./graphs/mc"
    else:
        save_dir = "./graphs/other"

    os.makedirs(save_dir, exist_ok=True)

    # Plot Win Rate
    plt.figure(figsize=(12, 6))
    plt.plot(df["Episode"], df["Win Rate (%)"], label="Win Rate (%)")
    plt.title("Agent Win Rate Over Episodes")
    plt.xlabel("Episode")
    plt.ylabel("Win Rate (%)")
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"win_rate_trend_{LOG_FILE.replace('.txt', '')}.png"))
    plt.close()

    # Plot Clicks per Game
    plt.figure(figsize=(12, 6))
    plt.plot(df["Episode"], df["Total Click Avg (c/g)"], label="Total Clicks", alpha=0.5)
    plt.plot(df["Episode"], df["Right Click Avg (c/g)"], label="Right Clicks (Flag)", color='red')
    plt.plot(df["Episode"], df["Left Click Avg (c/g)"], label="Left Clicks (Reveal)", color='green')
    plt.title("Average Clicks per Game Over Episodes")
    plt.xlabel("Episode")
    plt.ylabel("Average Clicks")
    plt.grid(True)
    plt.legend()
    plt.savefig(os.path.join(save_dir, f"click_behavior_trend_{LOG_FILE.replace('.txt', '')}.png"))
    plt.close()
    
    print(f"Graphs saved to {save_dir}")


In [15]:
if os.path.exists(LOG_FILE):
	df_metrics = parse_logs(LOG_FILE)
	
	# Save the data to a CSV for easy access
	csv_path = f"{LOG_FILE.replace('.txt', '')}.csv"
	df_metrics.to_csv(csv_path, index=False)
	print(f"Metrics successfully parsed and saved to {csv_path}")
	
	# Generate and save graphs
	generate_graphs(df_metrics)
	
	# Print the final data table to the console
	print("\n--- Final Metrics Table ---")
	print(df_metrics)
else:
	print(f"Error: {LOG_FILE} not found.")

Metrics successfully parsed and saved to a2c_logs_20251011_173431.csv
Graphs saved to ./graphs/a2c

--- Final Metrics Table ---
     Episode  Win Rate (%)  Wins  Losses  Left Click Avg (c/g)  \
0          1          0.00     0       1                  5.00   
1          2          0.00     0       2                  7.50   
2          3         33.33     1       2                  7.33   
3          4         25.00     1       3                  7.25   
4          5         20.00     1       4                  7.00   
..       ...           ...   ...     ...                   ...   
295      296         10.81    32     264                  7.70   
296      297         10.77    32     265                  7.69   
297      298         11.07    33     265                  7.68   
298      299         11.04    33     266                  7.67   
299      300         11.00    33     267                  7.66   

     Right Click Avg (c/g)  Total Click Avg (c/g)  
0                     5.00 